In [2]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy import misc, ndimage
import keras
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
import tarfile
from zipfile import ZipFile
import cv2

C:\Users\micka\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


ImportError: Traceback (most recent call last):
  File "C:\Users\micka\Anaconda3\lib\site-packages\tensorflow\python\pywrap_tensorflow.py", line 58, in <module>
    from tensorflow.python.pywrap_tensorflow_internal import *
  File "C:\Users\micka\Anaconda3\lib\site-packages\tensorflow\python\pywrap_tensorflow_internal.py", line 28, in <module>
    _pywrap_tensorflow_internal = swig_import_helper()
  File "C:\Users\micka\Anaconda3\lib\site-packages\tensorflow\python\pywrap_tensorflow_internal.py", line 24, in swig_import_helper
    _mod = imp.load_module('_pywrap_tensorflow_internal', fp, pathname, description)
  File "C:\Users\micka\Anaconda3\lib\imp.py", line 243, in load_module
    return load_dynamic(name, filename, file)
  File "C:\Users\micka\Anaconda3\lib\imp.py", line 343, in load_dynamic
    return _load(spec)
ImportError: DLL load failed: Le module spécifié est introuvable.


Failed to load the native TensorFlow runtime.

See https://www.tensorflow.org/install/install_sources#

In [ ]:
data_dir = '../data'
image_dir = '../imgs/images'
patients = os.listdir(data_dir)
images = os.listdir(image_dir)

In [3]:
patients_df = pd.read_csv(data_dir+'/Data_Entry_2017.csv')
patients_df.head()2

,Image Index,Finding Labels,Follow-up #,Patient ID,Patient Age,Patient Gender,View Position,OriginalImage[Width,Height],OriginalImagePixelSpacing[x,y],Unnamed: 11
0,00000001_000.png,Cardiomegaly,0,1,58,M,PA,2682,2749,0.143,0.143,NaN
1,00000001_001.png,Cardiomegaly|Emphysema,1,1,58,M,PA,2894,2729,0.143,0.143,NaN
2,00000001_002.png,Cardiomegaly|Effusion,2,1,58,M,PA,2500,2048,0.168,0.168,NaN
3,00000002_000.png,No Finding,0,2,81,M,PA,2500,2048,0.171,0.171,NaN
4,00000003_000.png,Hernia,0,3,81,F,PA,2582,2991,0.143,0.143,NaN


In [4]:
labels = ["Atelectasis", "Cardiomegaly", "Effusion", "Infiltration", "Mass", "Nodule", "Pneumonia", "Pneumothorax",
          "Consolidation", "Edema", "Emphysema", "Fibrosis", "Pleural_Thickening", "Hernia", "No Finding"]

In [5]:
# new_df = patients_df.drop(['Finding Labels'], axis = 1).copy()
new_df = patients_df.iloc[:,:7]
for label in labels:
    new_df[label] = new_df.apply(lambda row: label in row['Finding Labels'], axis = 1)
new_df.drop(['Finding Labels'], axis=1)
new_df.head()

,Image Index,Finding Labels,Follow-up #,Patient ID,Patient Age,Patient Gender,View Position,Atelectasis,Cardiomegaly,Effusion,...,Nodule,Pneumonia,Pneumothorax,Consolidation,Edema,Emphysema,Fibrosis,Pleural_Thickening,Hernia,No Finding
0,00000001_000.png,Cardiomegaly,0,1,58,M,PA,False,True,False,...,False,False,False,False,False,False,False,False,False,False
1,00000001_001.png,Cardiomegaly|Emphysema,1,1,58,M,PA,False,True,False,...,False,False,False,False,False,True,False,False,False,False
2,00000001_002.png,Cardiomegaly|Effusion,2,1,58,M,PA,False,True,True,...,False,False,False,False,False,False,False,False,False,False
3,00000002_000.png,No Finding,0,2,81,M,PA,False,False,False,...,False,False,False,False,False,False,False,False,False,True
4,00000003_000.png,Hernia,0,3,81,F,PA,False,False,False,...,False,False,False,False,False,False,False,False,True,False


In [6]:
bbox_df = pd.read_csv(data_dir+'/BBox_List_2017.csv')
bbox_df = bbox_df.iloc[:,:6]
bbox_df.head()

,Image Index,Finding Label,Bbox [x,y,w,h]
0,00013118_008.png,Atelectasis,225.084746,547.019217,86.779661,79.186441
1,00014716_007.png,Atelectasis,686.101695,131.543498,185.491525,313.491525
2,00029817_009.png,Atelectasis,221.830508,317.053115,155.118644,216.949153
3,00014687_001.png,Atelectasis,726.237288,494.951420,141.016949,55.322034
4,00017877_001.png,Atelectasis,660.067797,569.780787,200.677966,78.101695


In [7]:
x_box = bbox_df.loc[bbox_df['Image Index'] == '00013118_008.png']['Bbox [x']
x_box

0    225.084746
Name: Bbox [x, dtype: float64

In [8]:
bbox_df['Finding Label'].unique()

array(['Atelectasis', 'Cardiomegaly', 'Effusion', 'Infiltrate', 'Mass',
       'Nodule', 'Pneumonia', 'Pneumothorax'], dtype=object)

In [9]:
bbox_df['Image Index'].to_csv(data_dir+'/bbox_list.txt', index=False)  # Write All images with bbox

In [21]:
font = cv2.FONT_HERSHEY_SIMPLEX
bottomLeftCornerOfText = (10,500)
fontScale= 1
fontColor= (255,255,255)
lineType = 2
b,g,r,a = 0,255,0,0

In [25]:
# Check if an Image is a Bbox
with open(data_dir+'/bbox_list.txt', 'r') as bbox:
    b_images = bbox.read().splitlines()
    for b_image in b_images[:13]:
        if b_image in images:
            x_box = bbox_df.loc[bbox_df['Image Index'] == b_image]['Bbox [x']
            y_box = bbox_df.loc[bbox_df['Image Index'] == b_image]['y']
            w_box = bbox_df.loc[bbox_df['Image Index'] == b_image]['w']
            h_box = bbox_df.loc[bbox_df['Image Index'] == b_image]['h]']
            img = cv2.imread(image_dir+'/'+ b_image, cv2.IMREAD_COLOR)
#             cv2.putText(img, bbox_df.loc[bbox_df['Image Index'] == b_image]['Finding Label'], 
#                       (50,50), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (b,g,r), 1, cv2.LINE_AA)
            cv2.rectangle(img, (x_box, y_box), (w_box, h_box),(0,255,0), 5) # image/ coordinates/ color/width  
#             print('ok: ', b_image)
            
cv2.imshow('image', img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:

cv2.destroyAllWindows()